# LangChain Backend Experiment Number 3

The purpose of this experiment is to see if we can combine the chains in experiments one and two into one chain.

In [1]:
from __future__ import annotations
import openai
from langchain.agents import Tool
import os
import dotenv
dotenv.load_dotenv()
from langchain.chat_models import ChatOpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
import asyncio


from typing import Any, Dict, List, Optional

from pydantic import Extra

from langchain.schema import BaseLanguageModel
from langchain.callbacks.manager import (
    AsyncCallbackManagerForChainRun,
    CallbackManagerForChainRun,
)
from langchain.chains.base import Chain
from langchain.schema import BasePromptTemplate
import json

## Sample Project

In [2]:
project_details="""
We're building a comprehensive smart city solution, aiming to enhance the quality of urban life by using digital technology. This solution will have numerous interconnected components and features:

Smart Traffic Management System: Uses AI to analyze real-time traffic data, predict congestion, and control traffic lights to optimize traffic flow. It will also include a predictive maintenance system for traffic infrastructure.

Public Safety System: Incorporates facial recognition technology and predictive policing algorithms to ensure safety. This system will integrate with city surveillance cameras and police databases.

Waste Management System: Implements smart bins equipped with sensors to monitor waste levels and schedule optimal pick-up times.

Smart Energy Grid: Enables real-time monitoring of energy consumption and supply at an individual household level, while also integrating renewable energy sources.

Urban Farming: Incorporates vertical farming technologies, automated irrigation systems, and AI-powered pest prediction and prevention.

E-Government Portal: A central web portal for all government services, integrating multiple departmental systems and databases, complete with secure identity verification and multi-language support.

Public Transportation System: A fully automated scheduling and ticketing system for all city public transportation, including buses, subways, and bike rentals. It will use AI to optimize routes and schedules based on real-time data.

Environment Monitoring: Implementing IoT sensors across the city to monitor air and water quality in real time.
"""
project_technologies="""
The project will be developed using a range of technologies including, but not limited to, Python, TensorFlow, React.js, Node.js, PostgreSQL, Docker, Kubernetes, AWS, IoT protocols, and various data science and AI/ML libraries.

This system will have to comply with stringent regulations and standards for data security and privacy, including GDPR and ISO 27001. It will also require robust scalability, fault tolerance, and multi-region availability to cater to the entire city's population.

Additionally, the system will need to be user-friendly, accessible, and offer multi-language support to cater to the diverse user base in the city.
"""

## Setting Up the Backend Function

In [3]:
async def backend_func(
        inputs: Dict[str, Any],
        llm: BaseLanguageModel,
        advanced_llm: BaseLanguageModel,
    ) -> str:
        # Feature Extraction
        prompt = PromptTemplate(
            input_variables=["project_details", "project_technologies"],
            template="""
            Given the following project description and tech stack, identify and elaborate on the key backend features that would be necessary for development. The backend features should not involve any frontend components or styling. The backend features should be described in terms of capabilities.
            This project is a hackathon project. Break apart the features into MVP and additional features. The MVP should be the minimum features necessary to have a working prototype.
            Project description: {project_details}
            Technologies: {project_technologies}
            """
        )
        chain = LLMChain(llm=llm, prompt=prompt)
        backend_features = await chain.arun(inputs)
        
        # Specification Creation
        specification_prompt = PromptTemplate(
            input_variables=["backend_features", "project_technologies"],
            template="""
            Given the extracted backend features and the specified skills/technologies, create a detailed technical specification. 
            This specification should include the technologies to be used, the architecture, the different routes/endpoints, their inputs and outputs, and any potential hardware and startup costs.
            However, they should be split into two categories: MVP and additional features. The MVP should be the minimum features necessary to have a working prototype.
            You should ignore the technologies for the frontend and focus on the backend.
            Please also mention any other technical considerations.
            
            Backend Features: {backend_features}
            
            Project Technologies: {project_technologies}
            """
        )
        specification_chain = LLMChain(llm=llm, prompt=specification_prompt)
        specification = await specification_chain.arun({
            'backend_features': backend_features,
            'project_technologies': inputs['project_technologies']
        })
        
        approval_prompt = PromptTemplate(
            input_variables=["technical_specification", "aspect", "group_size", "group_experience"],
            template="""
            Given the developed technical specification, conduct a thorough review of the MVP Features only for any inconsistencies or issues. 
            Also, evaluate whether the MVP Features, can be realistically completed within the two day hackathon for {group_size} people, considering the complexity and the technology stack required.
            
            The MVP Features are specifically listed under the heading 'MVP Features'. 
            Please completely disregard any features or sections listed under 'Additional Features' or any similar headers.
            This specification is only for the {aspect} aspect of the project, and should not be evaluated for other aspects.

            Answer this question: Can the MVP Features be realistically completed within the two day hackathon for {group_size} people with this skill level: {group_experience}?
            Output only a json with keys 'approval' and 'comments'. 
            If yes, the value of 'approval' should be '1' and the value of 'comments' should be an empty string
            If not, the value of 'approval' should be '0' and the value of 'comments' should be a string with the issues and inconsistencies listed.

            Technical Specification: {technical_specification}
            
            Output only a json with keys 'approval' and 'comments'. 
            """
        )
        
        approval_chain = LLMChain(llm=advanced_llm, prompt=approval_prompt)
        approval = await approval_chain.arun({
            'technical_specification': specification,
            'aspect': 'backend',
            'group_size': inputs['group_size'],
            'group_experience': inputs['group_experience']
        })
        
        approvals_object = json.loads(approval)
        
        return_obj = {
            'approval': approvals_object['approval'],
            'comments': approvals_object['comments'],
            'features': backend_features,
            'specifications': specification
        }
        
        return json.dumps(return_obj)
        
        


## Testing this custom function

In [4]:
llm = ChatOpenAI(temperature=0, max_tokens=1000, model="gpt-3.5-turbo")
advanced_llm = ChatOpenAI(temperature=0, max_tokens=1000, model="gpt-4")

output = await backend_func(
    inputs = {
        'project_details': project_details,
        "project_technologies": project_technologies,
        "group_size": 4,
        "group_experience": "experienced"
    },
    llm=llm,
    advanced_llm=advanced_llm
)

In [5]:
json.loads(output)

{'approval': '0',
 'comments': 'The MVP features listed are quite complex and involve a wide range of technologies and skills. Even with an experienced team of 4 people, it would be challenging to complete all these features within a two-day hackathon. The Smart Traffic Management System and Public Safety System alone involve machine learning, data processing, IoT integration, and database management. Each of these areas could potentially take more than two days to implement effectively. It would be more realistic to focus on a subset of these features for the MVP.',
 'features': "MVP Backend Features:\n1. Smart Traffic Management System:\n- Real-time traffic data analysis: The backend should be able to collect and process real-time traffic data from various sources, such as traffic sensors and cameras.\n- Congestion prediction: Using AI/ML algorithms, the backend should be able to analyze the traffic data and predict congestion in different areas of the city.\n- Traffic light control:

In [6]:
async def backend_revision(
        inputs: Dict[str, Any],
        llm: BaseLanguageModel,
        advanced_llm: BaseLanguageModel,
    ) -> str:
        # Revision Prompt
        revision_prompt = PromptTemplate(
            input_variables=["original_features", "comments"],
            template="""
            Given the original features and the comments provided after the feasibility evaluation, modify the original features to make them simpler and more feasible for a two-day hackathon.
            The aim is to reduce the complexity and scope of the original features while preserving the core functionality of the system. 
            The comments provide specific areas of concern that should be addressed in the modified features.

            Original Features: {original_features}
            Comments: {comments}

            Based on this, please provide a simplified version of the original features that addresses the concerns mentioned in the comments.
            """
        )

        revision_chain = LLMChain(llm=llm, prompt=revision_prompt)
        backend_features = await revision_chain.arun({
            'original_features': inputs['features'],
            'comments': inputs['comments']
        })

        
        # Specification Creation
        specification_prompt = PromptTemplate(
            input_variables=["backend_features", "project_technologies"],
            template="""
            Given the extracted backend features and the specified skills/technologies, create a detailed technical specification. 
            This specification should include the technologies to be used, the architecture, the different routes/endpoints, their inputs and outputs, and any potential hardware and startup costs.
            However, they should be split into two categories: MVP and additional features. The MVP should be the minimum features necessary to have a working prototype.
            You should ignore the technologies for the frontend and focus on the backend.
            Please also mention any other technical considerations.
            
            Backend Features: {backend_features}
            
            Project Technologies: {project_technologies}
            """
        )
        specification_chain = LLMChain(llm=llm, prompt=specification_prompt)
        specification = await specification_chain.arun({
            'backend_features': backend_features,
            'project_technologies': inputs['project_technologies']
        })
        
        approval_prompt = PromptTemplate(
            input_variables=["technical_specification", "aspect", "group_size", "group_experience"],
            template="""
            Given the developed technical specification, conduct a thorough review of the MVP Features only for any inconsistencies or issues. 
            Also, evaluate whether the MVP Features, can be realistically completed within the two day hackathon for {group_size} people, considering the complexity and the technology stack required.
            
            The MVP Features are specifically listed under the heading 'MVP Features'. 
            Please completely disregard any features or sections listed under 'Additional Features' or any similar headers.
            This specification is only for the {aspect} aspect of the project, and should not be evaluated for other aspects.

            Answer this question: Can the MVP Features be realistically completed within the two day hackathon for {group_size} people with this skill level: {group_experience}?
            Output only a json with keys 'approval' and 'comments'. 
            If yes, the value of 'approval' should be '1' and the value of 'comments' should be an empty string
            If not, the value of 'approval' should be '0' and the value of 'comments' should be a string with the issues and inconsistencies listed.

            Technical Specification: {technical_specification}
            
            Output only a parsable one line json with keys 'approval' and 'comments'. 
            """
        )
        
        approval_chain = LLMChain(llm=advanced_llm, prompt=approval_prompt)
        approval = await approval_chain.arun({
            'technical_specification': specification,
            'aspect': 'backend',
            'group_size': inputs['group_size'],
            'group_experience': inputs['group_experience']
        })
        
        approvals_object = json.loads(approval)
        
        return_obj = {
            'approval': approvals_object['approval'],
            'comments': approvals_object['comments'],
            'features': backend_features,
            'specifications': specification
        }
        
        return json.dumps(return_obj)
        
        


In [7]:
for i in range(5):
    inputs = json.loads(output)
    inputs['project_technologies'] = project_technologies
    inputs['group_size'] = 4
    inputs['group_experience'] = 'experienced'
    output = await backend_revision(
        inputs = inputs,
        llm=llm,
        advanced_llm=advanced_llm
    )
    obj = json.loads(output)
    print(obj['features'])
    print(obj['approval'])
    print("=====================================")
    if(obj['approval'] == 1):
        break
        

Modified Features: MVP Backend Features:

1. Smart Traffic Management System:
- Real-time traffic data analysis: The backend should be able to collect and process real-time traffic data from various sources, such as traffic sensors.
- Congestion prediction: Using simple algorithms, the backend should be able to analyze the traffic data and provide basic congestion predictions.
- Traffic light control: The backend should have the capability to control traffic lights based on predefined rules to optimize traffic flow.
- Basic maintenance system: The backend should monitor the condition of traffic infrastructure and provide alerts for maintenance based on predefined thresholds.

2. Public Safety System:
- Facial recognition integration: The backend should integrate with facial recognition technology to identify individuals from surveillance cameras.
- Basic crime pattern analysis: The backend should analyze crime patterns based on historical data to identify potential crime hotspots.
- In